In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['sample_submission.csv', 'test.csv', 'train.csv']


In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sample_submission = pd.read_csv('../input/sample_submission.csv')

## **train and test dataset**

In [3]:
x_train = train.iloc[:,2:]
y_train = train['target']
x_test = test.iloc[:,1:]


In [4]:
print('train_feature shape:', x_train.shape)
print('train_target shape:', y_train.shape)
print('test_feature shape:', x_test.shape)

train_feature shape: (200000, 200)
train_target shape: (200000,)
test_feature shape: (200000, 200)


In [5]:
# augment sample to deal with the imbalanced dataset
def disarrange(a, axis=-1):
    """
    Shuffle `a` in-place along the given axis.

    Apply numpy.random.shuffle to the given axis of `a`.
    Each one-dimensional slice is shuffled independently.
    """
    b = a.swapaxes(axis, -1)
    # Shuffle `b` in-place along the last axis.  `b` is a view of `a`,
    # so `a` is shuffled in place, too.
    shp = b.shape[:-1]
    for ndx in np.ndindex(shp):
        np.random.shuffle(b[ndx])
    return

def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        disarrange(x1,axis=0)
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        disarrange(x1,axis=0)
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [6]:
#augment step
x_train_a, y_train_a = augment(x_train.values, y_train.values)

In [7]:
#switch augmented data from array to dataframe
#x_train_a_frame = pd.DataFrame(x_train_a)
#y_train_a_frame = pd.DataFrame(y_train_a)

In [8]:
#print("the train_set_feature shape:", x_train_a.shape)
#print("the train_set_labels shape:", y_train_a.shape)

## **NN **

In [9]:
# change the shape of the labels to 2.
def to_one_hot(labels, dimension = 2):
    results = np.zeros((len(labels),dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

In [10]:
#one-hot labels
y_train_vec = to_one_hot(y_train)
#y_train_a_vec = to_one_hot(y_train_a)
print('train_target shape:', y_train_vec.shape)

train_target shape: (200000, 2)


In [11]:
# random separate the train and vaild.
#agumented data make the overfitting

#rand_ind = np.random.permutation(420098)
#train_ind = rand_ind[0:400000]
#valid_ind = rand_ind[400000:420098]

rand_ind = np.random.permutation(200000)
train_ind = rand_ind[0:180000]
valid_ind = rand_ind[180000:200000]

x_valid_vec = x_train.iloc[valid_ind,:]
y_valid_vec = y_train_vec[valid_ind,:]

x_train_vec = x_train.iloc[train_ind,:]
y_train_vec = y_train_vec[train_ind,:]



In [12]:
from keras import models
from keras import layers
from keras import optimizers

Using TensorFlow backend.


In [13]:
#build NN model
model = models.Sequential()
model.add(layers.Dense(10, activation='relu', input_shape = (200,)))
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(48, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(2, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                2010      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                220       
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 48)                1008      
_________________________________________________________________
dense_4 (Dense)              (None, 128)               6272      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
__________

In [15]:
model.compile(optimizers.RMSprop(lr=2E-4),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [16]:
history = model.fit(x_train_vec,y_train_vec,
                  batch_size=32,epochs=50,
                  validation_data=(x_valid_vec,y_valid_vec))

Instructions for updating:
Use tf.cast instead.
Train on 180000 samples, validate on 20000 samples
Epoch 1/50
180000/180000 [==============================] - 25s 137us/step - loss: 0.3115 - acc: 0.8994 - val_loss: 0.2978 - val_acc: 0.8959
Epoch 2/50
180000/180000 [==============================] - 23s 127us/step - loss: 0.2821 - acc: 0.9019 - val_loss: 0.2881 - val_acc: 0.8992
Epoch 3/50
180000/180000 [==============================] - 23s 125us/step - loss: 0.2793 - acc: 0.9047 - val_loss: 0.2935 - val_acc: 0.9011
Epoch 4/50
180000/180000 [==============================] - 24s 132us/step - loss: 0.2854 - acc: 0.9060 - val_loss: 0.2689 - val_acc: 0.9030
Epoch 5/50
180000/180000 [==============================] - 23s 125us/step - loss: 0.2993 - acc: 0.9066 - val_loss: 0.2903 - val_acc: 0.9020
Epoch 6/50
180000/180000 [==============================] - 23s 125us/step - loss: 0.3479 - acc: 0.9065 - val_loss: 0.2982 - val_acc: 0.9038
Epoch 7/50
180000/180000 [=============================

In [17]:
prediction = pd.DataFrame(model.predict(x_test))
nn_result = np.array(prediction.columns[np.where(prediction!=0)[1]])

## **XBG**


In [18]:
from xgboost import XGBClassifier
import xgboost as xgb
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold


In [19]:
skf = StratifiedKFold(n_splits =4,random_state = 0,shuffle=False)
for train_index, test_index in skf.split(x_train_a,y_train_a):
    x_train_skf,x_val_skf = x_train_a[train_index],x_train_a[test_index]
    y_train_skf,y_val_skf = y_train_a[train_index],y_train_a[test_index]
    

In [20]:
clf = XGBClassifier(colsample_bylevel = 0.8015579071911014,
        colsample_bytree = 0.44364889457651413,
        gamma = 3.811128976537413e-05,
        learning_rate = 0.2700390206185342,
        max_delta_step = 18,
        max_depth = 36,
        min_child_weight = 2,
        n_estimators = 83,
        reg_alpha = 1.5057560255472018e-06,
        reg_lambda = 0.08186810622382998,
        scale_pos_weight = 0.02900459363415458,
        subsample = 0.8835665823899177)
clf.fit(x_train_skf,y_train_skf,eval_set = [(x_train_skf,y_train_skf),(x_val_skf,y_val_skf)],
       eval_metric='auc',early_stopping_rounds=10)
evals_results = clf.evals_result()

[0]	validation_0-auc:0.611704	validation_1-auc:0.611061
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.684828	validation_1-auc:0.681325
[2]	validation_0-auc:0.7181	validation_1-auc:0.712595
[3]	validation_0-auc:0.742641	validation_1-auc:0.732268
[4]	validation_0-auc:0.750097	validation_1-auc:0.733779
[5]	validation_0-auc:0.765626	validation_1-auc:0.743994
[6]	validation_0-auc:0.781835	validation_1-auc:0.752629
[7]	validation_0-auc:0.799561	validation_1-auc:0.760757
[8]	validation_0-auc:0.81259	validation_1-auc:0.764111
[9]	validation_0-auc:0.835407	validation_1-auc:0.774806
[10]	validation_0-auc:0.846516	validation_1-auc:0.776946
[11]	validation_0-auc:0.857934	validation_1-auc:0.784489
[12]	validation_0-auc:0.864364	validation_1-auc:0.789417
[13]	validation_0-auc:0.869926	validation_1-auc:0.793323
[14]	validation_0-auc:0.875611	validation_1-auc:0.797371
[

In [21]:
x_test_a = np.array(x_test)
x_test_a

array([[ 11.0656,   7.7798,  12.9536, ...,  10.72  ,  15.4722,  -8.7197],
       [  8.5304,   1.2543,  11.3047, ...,   9.8714,  19.1293, -20.976 ],
       [  5.4827, -10.3581,  10.1407, ...,   7.0618,  19.8956, -23.1794],
       ...,
       [ 11.636 ,   2.2769,  11.2074, ...,   9.1933,  11.7905, -22.2762],
       [ 13.5745,  -0.5134,  13.6584, ...,   8.1079,   8.7735,  -0.2122],
       [ 10.4664,   1.807 ,  10.2277, ...,  10.3378,  14.334 ,  -7.7094]])

In [22]:
xgb_result = clf.predict(x_test_a)

## **LGB**

In [23]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [24]:
from sklearn.model_selection import KFold
num_folds = 11
features = [c for c in x_train.columns if c not in ['ID_code', 'target']]
#target = x_train['target']

folds = KFold(n_splits=num_folds, random_state=2319)
#oof = np.zeros(len(x_train))
#getVal = np.zeros(len(x_train))
lgb_result = np.zeros(len(y_train))
#feature_importance_df = pd.DataFrame()

In [25]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.041,
    'learning_rate': 0.0083,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': -1
}

for fold_, (trn_idx, val_idx) in enumerate(folds.split(x_train.values, y_train.values)):
    
    X_train_lgb, y_train_lgb = x_train.iloc[trn_idx], y_train.iloc[trn_idx]
    X_valid_lgb, y_valid_lgb = x_train.iloc[val_idx], y_train.iloc[val_idx]
    
    X_tr, y_tr = augment(X_train_lgb.values, y_train_lgb.values)
    X_tr = pd.DataFrame(X_tr)
    
    print("Fold idx:{}".format(fold_ + 1))
    trn_data = lgb.Dataset(X_tr, label=y_tr)
    val_data = lgb.Dataset(X_valid_lgb, label=y_valid_lgb)
    
    clf = lgb.train(param, trn_data, 1000000, valid_sets = [trn_data, val_data], verbose_eval=5000, early_stopping_rounds = 4000)
    #oof[val_idx] = clf.predict(x_train.iloc[val_idx], num_iteration=clf.best_iteration)
    #getVal[val_idx]+= clf.predict(x_train.iloc[val_idx], num_iteration=clf.best_iteration) / folds.n_splits
    
    #fold_importance_df = pd.DataFrame()
    #fold_importance_df["feature"] = features
    #fold_importance_df["importance"] = clf.feature_importance()
    #fold_importance_df["fold"] = fold_ + 1
    #feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    lgb_result += clf.predict(x_test, num_iteration=clf.best_iteration) / folds.n_splits

Fold idx:1
Training until validation scores don't improve for 4000 rounds.
[5000]	training's auc: 0.912229	valid_1's auc: 0.896459
[10000]	training's auc: 0.922006	valid_1's auc: 0.900289
[15000]	training's auc: 0.929514	valid_1's auc: 0.901064
[20000]	training's auc: 0.936353	valid_1's auc: 0.901116
Early stopping, best iteration is:
[17484]	training's auc: 0.932962	valid_1's auc: 0.901285
Fold idx:2
Training until validation scores don't improve for 4000 rounds.
[5000]	training's auc: 0.912059	valid_1's auc: 0.894891
[10000]	training's auc: 0.921827	valid_1's auc: 0.898231
[15000]	training's auc: 0.929416	valid_1's auc: 0.89854
Early stopping, best iteration is:
[13985]	training's auc: 0.927946	valid_1's auc: 0.898677
Fold idx:3
Training until validation scores don't improve for 4000 rounds.
[5000]	training's auc: 0.912995	valid_1's auc: 0.889396
[10000]	training's auc: 0.922797	valid_1's auc: 0.892885
[15000]	training's auc: 0.930287	valid_1's auc: 0.8932
Early stopping, best iterat

**result**

In [26]:
sample_submission['target'] = 0.3*nn_result + 0.4*xgb_result + 0.4*lgb_result
sample_submission.to_csv('submission.csv',index = False)

ValueError: operands could not be broadcast together with shapes (400000,) (200000,) 